In [137]:
import torch
from models import skipgram
import pandas as pd
import os
from util import artifacts

import importlib
importlib.reload(artifacts)

<module 'util.artifacts' from '/Users/cameron/Documents/mlx/hackernews/util/artifacts.py'>

In [150]:
loaded_artifacts = artifacts.load_artifacts()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [ ]:
state_dict = loaded_artifacts['state_dict']
vocab = loaded_artifacts['vocab'].set_index('token')

model = skipgram.Model(vocab.size + 1, skipgram.EMBEDDING_DIM)

def getIdFromToken(token: str):
    try:
        return int(vocab.at[token, 'id'])
    except:
        return int(vocab.at['unk', 'id'])

# #find the n most similar words
def most_similar(word, n=5):
    word_idx = getIdFromToken(word)
    A =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
    word_similarities = []
    for token, id in vocab.iterrows():
        word_idx = getIdFromToken(token)
        B =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
        cosine_similarity = torch.nn.functional.cosine_similarity(A, B, dim=1)
        word_similarities.append((token, cosine_similarity.item()))
    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    return word_similarities[:n]

# girl_id = getIdFromToken('girl')
# girl = model.embeddings.weight[girl_id].unsqueeze(0)
# boy_id = getIdFromToken('boy')
# boy = model.embeddings.weight[boy_id].unsqueeze(0)
# torch.nn.functional.cosine_similarity(girl, boy, dim=1)

most_similar('javascript')

[('javascript', 1.0),
 ('java', 0.5587868094444275),
 ('django', 0.557133674621582),
 ('pdf', 0.5392530560493469),
 ('nigerias', 0.5322160720825195)]